# Normalize place names
Compare packages for normalizing place names

- What processing steps were applied to create `city` and `state` columns?
- What are the alternate columns?
- Which file contains the highest quality output?

Packages:
1. [geocoder](https://geocoder.readthedocs.io/providers/GeoNames.html) uses GeoNames API
2. [reconciler](https://pypi.org/project/reconciler/) uses Wikidata by default and returns match score

To do: 
- [ ] preserve original record id
- [ ] improve state name matching
- [ ] evaluate quality of both approaches

In [1]:
from tqdm import tqdm
import re
import pandas as pd
pd.set_option("display.max_columns", None)

In [2]:
df = pd.read_csv('../merged_ocr_20230119.csv',index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25744 entries, 0 to 25743
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   file              25744 non-null  object 
 1   backcard_flag     25744 non-null  int64  
 2   city              25197 non-null  object 
 3   state             25158 non-null  object 
 4   lname             25444 non-null  object 
 5   p1_name           25336 non-null  object 
 6   p2_name           15960 non-null  object 
 7   amount            24324 non-null  object 
 8   agency            25609 non-null  object 
 9   agency_city       24384 non-null  object 
 10  agency_state      24345 non-null  object 
 11  ref_flag          1024 non-null   object 
 12  id                25562 non-null  object 
 13  status            25566 non-null  object 
 14  other             380 non-null    object 
 15  city_alt          19762 non-null  object 
 16  state_alt         19642 non-null  object

Drop records that don't have a city and state because we won't be able to match these externally

In [3]:
print(f"Removing {len(df[(df.city.isnull()) & (df.state.isnull())])} records without CITY or STATE")
df = df[(df.city.notnull()) & (df.state.notnull())]

Removing 547 records without CITY or STATE


Filter to unique pairs of `city` and `state` to reduce total queries

In [4]:
print(f"Removing {len(df.duplicated(subset=['city', 'state']))} duplicated CITY and STATE records")
df = df.drop_duplicates(subset=['city', 'state'])
df = df[['city', 'state']]

Removing 25158 duplicated CITY and STATE records


Clean up state names by matching state names to abbreviations with [dictionary](https://gist.github.com/rogerallen/1583593)

In [5]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

Replace state name with standard abbreviation

In [6]:
def replace_states(x):
    if len(x) == 2:
        for a,n in states.items():
            if len(n.split()) == 2:
                if "".join([c[0] for c in n.split()]).lower() == x.lower():
                    return a.upper()
    new_rx = re.compile(r"\w*".join([ch for ch in x]), re.I)
    for a,n in states.items():
        if new_rx.match(n):
            return a.upper()

# states = {state: abbrev for state, abbrev in states.items()}
# df['state_corrected'] = df['state'].replace(states)     
        
df['state_corrected'] = df['state'].apply(lambda x: replace_states(x))
df['country'] = 'United States' # add country info for disambiguation
df

,city,state,state_corrected,country
0,Wichita,Kansas,None,United States
2,Detroit,Mich.,None,United States
3,Tulsa,Oklahoma,None,United States
4,Atlanta,Ga.,GU,United States
5,Chicago,Ill.,None,United States
...,...,...,...,...
25733,South Bend,Indisna,None,United States
25736,Bast Alton,Illinois,None,United States
25738,Erie,Pa.,PA,United States
25742,Perth Amboy,N. d.,None,United States


Sample random place names from dataframe to look up

In [7]:
sample = df.sample(5)
sample[['city','state','state_corrected','country']]

,city,state,state_corrected,country
3451,Stillwater,Okla.,None,United States
8458,Dresden,Tenn.,None,United States
9514,Cheyenne,Wyo.,None,United States
3636,lufkin,Texas,TX,United States
23040,Lower Penns Neck Twp.,N. Jd.,None,United States


## Geocoder (uses GeoNames)
- clean up state columns further to improve matching

In [8]:
import geocoder

In [9]:
def geocode(q):
    g = geocoder.geonames(q, fuzzy=0.9, featureClass='P', key='slafia')
    result=[g.address,g.state,g.country]
    df = pd.DataFrame([result])
    df.columns =['address','state','country']
    return df

In [10]:
geocode(sample.city.iloc[0])

,address,state,country
0,Stillwater,New York,United States


In [11]:
# test_df = df[0:10]

frames = []

for i in tqdm(range(len(df))):
    q = df.city.iloc[i]
    result = geocode(q)
    frames.append(result)

geocoder_result = pd.concat(frames)
geocoder_result.to_csv("../geocoder_result.csv",index=False) 
geocoder_result

100%|██████████| 4991/4991 [28:05<00:00,  2.96it/s]


,address,state,country
0,Wichita,Kansas,United States
0,Detroit,Michigan,United States
0,Tulsa,Oklahoma,United States
0,Atlanta,Georgia,United States
0,Chicago,Illinois,United States
...,...,...,...
0,None,None,None
0,None,None,None
0,None,None,None
0,None,None,None


## Reconciler (uses Wikidata)
- use property mappings to control matching (city, state, country)

In [12]:
from reconciler import reconcile

In [13]:
reconcile(sample['city'], type_id="Q515", property_mapping={"P17": sample["country"]})

100%|██████████| 1/1 [00:03<00:00,  3.78s/it]


,description,id,match,name,score,type,type_id,input_value
0,"city in Oklahoma, United States",Q852714,False,Stillwater,100.0,city in the United States,Q1093829,Stillwater
1,village in the U.S. state Kansas,Q737444,True,Dresden,100.0,city in the United States,Q1093829,Dresden
2,"capital of Wyoming, United States and seat of ...",Q39042,False,Cheyenne,100.0,city,Q515,Cheyenne
3,"city in Texas, USA",Q975590,True,Lufkin,100.0,city in the United States,Q1093829,lufkin
4,NaN,NaN,False,NaN,NaN,NaN,NaN,Lower Penns Neck Twp.


In [14]:
reconciler_result = reconcile(df['city'], type_id="Q515", property_mapping={"P17": sample["country"]})
reconciler_result.to_csv("../reconciler_result.csv",index=False) 
reconciler_result

100%|██████████| 270/270 [30:01<00:00,  6.67s/it]


,description,id,match,name,score,type,type_id,input_value
0,"city in Sedgwick County, Kansas, United States",Q49266,False,Wichita,89.142857,city in the United States,Q1093829,Wichita
1,"city in and county seat of Wayne County, Michi...",Q12439,False,Detroit,89.142857,city in the United States,Q1093829,Detroit
2,"city in Oklahoma, United States and county sea...",Q44989,False,Tulsa,89.142857,city in the United States,Q1093829,Tulsa
3,"capital city of Georgia, United States",Q23556,False,Atlanta,89.142857,state or insular area capital of the United St...,Q21518270,Atlanta
4,"city and county seat of Cook County, Illinois,...",Q1297,False,Chicago,89.142857,city of Illinois,Q110071194,Chicago
...,...,...,...,...,...,...,...,...
2686,NaN,NaN,False,NaN,NaN,NaN,NaN,Zemke
2687,NaN,NaN,False,NaN,NaN,NaN,NaN,EZ. St. Louis
2688,NaN,NaN,False,NaN,NaN,NaN,NaN,Murphysbore
2689,"historic site in Denby Drayton Plains, Michigan",Q24061822,False,Drayton Plains State Fish Hatchery,20.714286,[],NaN,Drayton Plains
